In [84]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [85]:
symbol = "INFY"

def get_historical_stock_price(symbol, to_date = datetime.today().strftime("%d-%m-%Y"), 
                                   from_date = (datetime.today() - timedelta(days = 367)).strftime("%d-%m-%Y")):
    df = equity_history(symbol, "EQ", from_date, to_date)
    df = df.drop(columns = ['_id','CH_SYMBOL','CH_SERIES','CH_MARKET_TYPE', 'CH_LAST_TRADED_PRICE',
                                    'CH_PREVIOUS_CLS_PRICE','CH_TOTAL_TRADES','TIMESTAMP','createdAt',
                                    'updatedAt','__v','VWAP','mTIMESTAMP','CA'])
    df.columns = ['high', 'low', 'open', 'close', 'volume', 'total_traded_value', 
                  'high_52_week', 'low_52_week', 'ISIN', 'date']
    return df.set_index('date').sort_index()

In [86]:
df = get_historical_stock_price(symbol)
df['volume_sma_10D'] = df['volume'] > df['volume'].rolling(10).mean()

In [87]:
def avg_crossovers(fast_series, slow_series):
    max_column = pd.concat([fast_series, slow_series], axis = 1).idxmax(axis = 1)
    return np.where(np.logical_and(max_column.notnull(), max_column != max_column.shift(1)), 
                        np.where(fast_series.name == max_column, "Buy", "Sell"), "")

In [88]:
ema_days = [9, 12, 21, 25, 26, 50, 100]
for i in ema_days:
    df[f'close_ema_{i}D'] = df['close'].ewm(span = i, min_periods = i).mean()

In [89]:
df['MACD'] = df['close_ema_12D'] - df['close_ema_26D']
df['MACD_signal_line'] = df['MACD'].ewm(span = 9, min_periods = 9).mean()
df['MACD_indicator'] = avg_crossovers(df['MACD'], df['MACD_signal_line'])
df['close_ema_crossover_9_21'] = avg_crossovers(df['close_ema_9D'], df['close_ema_21D'])
df['close_ema_crossover_25_50'] = avg_crossovers(df['close_ema_25D'], df['close_ema_50D'])
df['close_ema_crossover_50_100'] = avg_crossovers(df['close_ema_50D'], df['close_ema_100D'])

In [105]:
def rsi_indicator(close_price, period = 14):
    close_price_diff = close_price.diff(1)
    avg_gain = close_price_diff.where(close_price_diff > 0, 0).ewm(alpha = 1 / period, min_periods = period, adjust = False).mean()
    avg_loss = - close_price_diff.where(close_price_diff < 0, 0).ewm(alpha = 1 / period,min_periods = period, adjust = False).mean()
    RSI = 100 - (100/(1 + avg_gain / avg_loss))
    return RSI

In [106]:
rsi_indicator(df['close'])

date
2021-09-13          NaN
2021-09-14          NaN
2021-09-15          NaN
2021-09-16          NaN
2021-09-17          NaN
                ...    
2022-09-07    37.748399
2022-09-08    42.619623
2022-09-09    50.749696
2022-09-12    55.420837
2022-09-13    56.986715
Name: close, Length: 257, dtype: float64